# Paso 4: Data mining

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Cargamos datos tras paso 2
# tracks = pd.read_csv(r'E:\Documentos\Github\proyecto-analisis-de-datos-1INF03\datos\tracks.csv')
# dfunion = pd.read_csv(r'F:\2022-1\Analisis de datos\Tarea Academica\archive\tracks.csv')
dfunion = pd.read_csv("../datos/dfunion_post_imp.csv")

## Se parar en variable dependiente y variable independiente

In [2]:
y = dfunion[["list_pop"]]
x = dfunion[['duration_ms', 'danceability', 'energy', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'explicit', 'mode', 'Name_Length',
       'words_name', 'release_year', 'release_month', 'release_day',
       'release_trim', 'time_signature', 'list_key']]

## Sample splitting

In [3]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=100, stratify=y)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=200, stratify=y_train)

## Balanceo de Datos

In [4]:
pd.DataFrame(y_train).value_counts(normalize=True)*100

list_pop
0           73.128421
1           26.871579
dtype: float64

Nuestra variable no está tan desbalanceada

In [5]:
nas = pd.DataFrame(x_train.isnull().sum()).sort_values(0,ascending=False)
nas.columns = ['nas']
nas['nas%'] = round(nas['nas']/x_train.shape[0], 2)
nas 

,nas,nas%
duration_ms,0,0.0
danceability,0,0.0
time_signature,0,0.0
release_trim,0,0.0
release_day,0,0.0
release_month,0,0.0
release_year,0,0.0
words_name,0,0.0
Name_Length,0,0.0
mode,0,0.0


No tenemos valores nulos, asi que podemos generar datos balanceados

In [6]:
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from collections import Counter 

os_us = SMOTETomek(random_state=2022, sampling_strategy='all')

x_train_smote, y_train_smote = os_us.fit_resample(x_train, y_train)

#x_train_all_smote, y_train_all_smote = os_us.fit_resample(x_train, y_train)

In [10]:
print(y_train.value_counts(normalize=True)*100) #antes del balanceo
print(y_train.value_counts(normalize=True)*100) 

print(y_train_smote.value_counts(normalize=True)*100) #luego de aplicar la tecnica smote
print(y_train_smote.value_counts(normalize=True)*100)

list_pop
0           73.128421
1           26.871579
dtype: float64
list_pop
0           73.128421
1           26.871579
dtype: float64
list_pop
0           50.0
1           50.0
dtype: float64
list_pop
0           50.0
1           50.0
dtype: float64


## Árbol de clasificación CART

### Generamos arboles de decision con datos balanceados y sin balancear

Elegimos el modelo

In [13]:
from sklearn.tree import DecisionTreeClassifier

# Creacion del modelo 
modelo1 = DecisionTreeClassifier(criterion="gini", random_state=2022)
modelo1b = DecisionTreeClassifier(criterion="gini", random_state=2022)
# Indicador gini ayuda a medir la calidad de la división de los datos

Entrenamos el modelo 

In [14]:
# Entrenamiento del modelo 
modelo1.fit(x_train, y_train)

DecisionTreeClassifier(random_state=2022)

In [15]:
# Entrenamiento del modelo 
modelo1b.fit(x_train_smote, y_train_smote)

DecisionTreeClassifier(random_state=2022)

Realizamos predicciones con datos de test

In [16]:
# Realizamos predicciones con datos de test
y_predict_modelo1 = modelo1.predict(x_test)
y_predict_modelo1

array([0, 0, 0, ..., 0, 1, 0], dtype=int64)

In [17]:
# Realizamos predicciones con datos de test
y_predict_modelo1b = modelo1b.predict(x_test)
y_predict_modelo1b

array([0, 0, 1, ..., 0, 1, 0], dtype=int64)

FALTA GRID SEARCH

In [18]:
# Se calcula el accuracy
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

accuracy_score(y_test, y_predict_modelo1) 

0.7337351944412244

In [19]:
# Se calcula el accuracy
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

accuracy_score(y_test, y_predict_modelo1b) 

0.7195450423113378

In [20]:
from sklearn.model_selection import cross_val_score

scores_res = cross_val_score(modelo1, x_train, y_train, cv=10)
print("Accuracy de los 10 folds:", scores_res)
print("Accuracy Mean: {:.3f} (std: {:.3f})".format(scores_res.mean(), scores_res.std()) )

Accuracy de los 10 folds: [0.73209686 0.73146044 0.73378591 0.73454358 0.73378591 0.73572554
 0.73230088 0.73154322 0.73036126 0.72936113]
Accuracy Mean: 0.732 (std: 0.002)


MATRIZ DE CONFUSION

In [59]:
y_test.to_numpy()

array([[0],
       [0],
       [1],
       ...,
       [0],
       [1],
       [1]], dtype=int64)

In [58]:
y_predict_modelo1

array([0, 0, 0, ..., 0, 1, 0], dtype=int64)

In [60]:
# Confusion matrix
print(pd.crosstab(y_test.to_numpy(), y_predict_modelo1))

MemoryError: Unable to allocate 160. GiB for an array with shape (146651, 146651) and data type int64

In [16]:
# Se genera el arbol de clasificacion 
target = list(y_test['list_pop'].unique())
feature_names = list(x_train.columns)

Falta graficar arbol

In [ ]:
from sklearn import tree
from sklearn.tree import graphviz
graphviz.Source(tree.export_graphviz(modelo, 
                                     feature_names = feature_names, 
                                     class_names = target, 
                                     filled=True, rounded=True))  

Guardamos el modelo

In [ ]:
import pickle 
pickle.dump(modelo1, open('modelo1', 'wb')) 

Realizamos predicciones en el modelo 1 

In [ ]:
import_modelo1 = pickle.load(open('modelo1', 'rb'))
result = import_modelo1.predict(x_test_num)
pd.DataFrame(result).value_counts() 